In [1]:
!kaggle datasets download -d parvmodi/english-to-hindi-machine-translation-dataset

Dataset URL: https://www.kaggle.com/datasets/parvmodi/english-to-hindi-machine-translation-dataset
License(s): CC0-1.0
100% 1.04G/1.04G [00:15<00:00, 88.2MB/s]
100% 1.04G/1.04G [00:15<00:00, 73.7MB/s]


In [2]:
!unzip english-to-hindi-machine-translation-dataset.zip

Archive:  english-to-hindi-machine-translation-dataset.zip
  inflating: train.en                
  inflating: train.hi                


In [57]:
filepath_en = 'train.en'
filepath_hi = 'train.hi'

with open(filepath_en, encoding='utf-8') as file:
    lines_en = file.readlines()
print(len(lines_en))
print(lines_en[:5])

with open(filepath_hi, encoding='utf-8') as file:
    lines_hi = file.readlines()

print(len(lines_hi))
print(lines_hi[:5])

10125706
["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles\n", 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.\n', 'The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.\n', 'Mithali To Anchor Indian Team Against Australia in ODIs\n', 'After the assent of the Honble President on 8thSeptember, 2016, the 101thConstitutional Amendment Act, 2016 came into existence\n']
10125706
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।\n', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है\n', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प

In [58]:
TOTAL_SENTENCES = 200000
english_sentences = lines_en[:TOTAL_SENTENCES]
hindi_sentences = lines_hi[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

print(english_sentences[:5])
print("================================")
print(hindi_sentences[:5])

["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles", 'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.', 'mithali to anchor indian team against australia in odis', 'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence']
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले मे

In [59]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 258.0
97th percentile length English: 267.0


In [60]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\' ', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

hindi_vocabulary = [START_TOKEN, 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ',
    'क', 'ख', 'ग', 'घ', 'ङ','ड़', 'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
    'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ',
    'ं', 'ः', 'ँ', '्', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '|',
    ':', '<', '=', '>', '?', '@',
    '[', '\' ', ']', '^', '_', '`',
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','\u200d','।', 'ृ',
    PADDING_TOKEN, END_TOKEN
]

print(len(english_vocabulary))

71


In [61]:
max_sequence_length = 250

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")


Number of sentences: 200000
Number of valid sentences: 151260


In [62]:
# for i in 'शख्स':
  # print(i)


# hindi_sentences = [
#     'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
#     'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
#     'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
#     'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
#     '8 सितम्‍बर, 2016 को माननीय राष्ट्रपति की स्‍वीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्व में आया'
# ]

# Define the provided vocabulary set
hindi_vocabulary_set = set(hindi_vocabulary)

# Combine all sentences into one string
combined_sentences = ''.join(hindi_sentences)

# Find all unique characters in the combined sentences
unique_chars_in_sentences = set(combined_sentences)

# Find characters in the sentences that are not in the vocabulary
missing_chars = unique_chars_in_sentences - hindi_vocabulary_set

print("Characters in the sentences not in the vocabulary:", missing_chars)


Characters in the sentences not in the vocabulary: {'p', 'V', 'ֹ', 'ы', 'ऩ', '州', 'я', 'ᐃ', 'প', 's', 'a', 'é', 'כ', '้', 'ז', 'ಮ', 'ො', '\u2008', 'ঙ', 'к', 'ख़', 'ප', '™', 'ᡠ', 'ං', 'n', '⁰', 'μ', 'স', 'ॆ', 'א', 'ó', 'ּ', 'ص', '»', '❏', 'ऌ', 'ط', '¸', 'ظ', 'ِ', 'ী', '˝', 'ර', 'ḷ', 'ٕ', 'ঘ', 'К', 'ě', '⊆', '੍', 'ុ', 'ॉ', 'W', 'ন', '্', '්', '׃', '骨', 'у', 'コ', '車', 'с', 'ٓ', 'ل', 'ு', 'â', 'ι', 'ֶ', '°', 'ව', 'Ō', 'ַ', 'š', 'ミ', '\ue5a0', 'L', 'េ', 'R', 'ی', '鲁', 'Ю', 'Ż', 'τ', '에', 'ॐ', '℃', '\uf514', '한', 'ਵ', 'С', 'ឃ', 'า', '日', 'á', '£', 'ع', 'α', '▫', '}', '章', 'ş', 'δ', '木', '\ufeff', 'ο', '⋅', 'Р', '²', 'ה', 'ー', 'ę', 'O', 'ò', 'ن', 'ہ', 'ש', 'ᠠ', 'ก', 'ุ', 'y', '្', 'வ', 'м', '́', 't', 'ǔ', '女', '经', 'ิ', 'K', '್', 'พ', 'ம', 'ί', 'ऑ', 'ঔ', 'ల', 'و', '\uf0d8', 'জ', '்', 'o', 'ح', '′', 'ও', 'চ', '☞', 'ִ', 'ң', 'р', 'ت', 'ಿ', 'ব', 'υ', '਼', '‘', '¹', '휴', 'ग़', 'ல', 'ε', '॰', 'М', '\ue205', 'י', 'з', '’', 'h', '唐', '☺', 'm', 'ᠴ', 'ᓗ', 'م', 'ŋ', 'Z', 'J', 'ऍ', 'ʔ', 'ς', 'ا', '¡', 'র'

In [63]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}
print(english_to_index)

{'<START>': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, '<': 28, '=': 29, '>': 30, '?': 31, '@': 32, '[': 33, "' ": 34, ']': 35, '^': 36, '_': 37, '`': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64, '{': 65, '|': 66, '}': 67, '~': 68, '<PADDING>': 69, '<END>': 70}


In [64]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras import layers
import numpy as np

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_seq_len):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

    def call(self, inputs):
        pos = np.arange(self.max_seq_len)[:, np.newaxis]
        i = np.arange(self.d_model)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.d_model))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.constant(pos_encoding, dtype=tf.float32)

class SentenceEmbedding(tf.keras.Model):
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super(SentenceEmbedding, self).__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = Dropout(0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token, end_token):
        def tokenize(sentence, start_token, end_token):
            sentence_word_indices = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indices.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indices.append(self.language_to_index[self.END_TOKEN])
            sentence_word_indices.extend([self.language_to_index[self.PADDING_TOKEN]] * (self.max_sequence_length - len(sentence_word_indices)))
            return tf.convert_to_tensor(sentence_word_indices, dtype=tf.int32)

        tokenized = [tokenize(sentence, start_token, end_token) for sentence in batch]
        return tf.stack(tokenized)

    def call(self, x, start_token, end_token):
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder(x)
        x = self.dropout(x + pos)
        return x

In [65]:
def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_english_sentences = []
valid_hindi_sentences = []
for index in valid_sentence_indicies:
    valid_english_sentences.append(english_sentences[index])
    valid_hindi_sentences.append(hindi_sentences[index])

def batch_tokenize(language_to_index, batch, start_token, end_token, START_TOKEN, END_TOKEN, PADDING_TOKEN, max_sequence_length):
    def tokenize(sentence, start_token, end_token):
        sentence_word_indices = [language_to_index[token] for token in list(sentence)]
        if start_token:
            sentence_word_indices.insert(0, language_to_index[START_TOKEN])
        if end_token:
            sentence_word_indices.append(language_to_index[END_TOKEN])
        sentence_word_indices.extend([language_to_index[PADDING_TOKEN]] * (max_sequence_length - len(sentence_word_indices)))
        return tf.convert_to_tensor(sentence_word_indices, dtype=tf.int32)

    tokenized = [tokenize(sentence, start_token, end_token) for sentence in batch]
    return tf.stack(tokenized)

x = batch_tokenize(language_to_index = english_to_index, batch = valid_english_sentences[:10], start_token = True, end_token = True,
                     START_TOKEN= START_TOKEN, END_TOKEN= END_TOKEN, PADDING_TOKEN=PADDING_TOKEN, max_sequence_length= max_sequence_length)

print(x)

tf.Tensor(
[[ 0 46 53 ... 69 69 69]
 [ 0 61 46 ... 69 69 69]
 [ 0 58 46 ... 69 69 69]
 ...
 [ 0 48 46 ... 69 69 69]
 [ 0 39 56 ... 69 69 69]
 [ 0 40 56 ... 69 69 69]], shape=(10, 250), dtype=int32)


In [66]:
import tensorflow as tf

d_model = 512

sentence_embedding = SentenceEmbedding(
    max_sequence_length=max_sequence_length,
    d_model=d_model,
    language_to_index=english_to_index,
    START_TOKEN=START_TOKEN,
    END_TOKEN=END_TOKEN,
    PADDING_TOKEN=PADDING_TOKEN
)

output = sentence_embedding(valid_english_sentences[:10], start_token=True, end_token=True)

print(output)

tf.Tensor(
[[[ 0.04724257  1.0271591  -0.01443849 ...  0.9656585  -0.01987906
    1.0488182 ]
  [ 0.8396481   0.5532805   0.8504856  ...  1.0116874  -0.00496451
    1.0300165 ]
  [ 0.86095643 -0.389178    0.98013884 ...  0.9905458   0.04022251
    0.9985762 ]
  ...
  [ 0.9552097  -0.38166624 -0.42940688 ...  1.034032    0.00436316
    1.0489395 ]
  [ 0.21315461 -0.9889139   0.49825913 ...  1.0340291   0.00446679
    1.0489368 ]
  [-0.6987607  -0.69259334  1.0323044  ...  1.0340263   0.00457042
    1.0489341 ]]

 [[ 0.04724257  1.0271591  -0.01443849 ...  0.9656585  -0.01987906
    1.0488182 ]
  [ 0.8472885   0.5511036   0.85813016 ...  1.0285046  -0.03072091
    1.030985  ]
  [ 0.9074746  -0.40316865  0.96504414 ...  1.0116874  -0.00486085
    1.0300165 ]
  ...
  [ 0.9552097  -0.38166624 -0.42940688 ...  1.034032    0.00436316
    1.0489395 ]
  [ 0.21315461 -0.9889139   0.49825913 ...  1.0340291   0.00446679
    1.0489368 ]
  [-0.6987607  -0.69259334  1.0323044  ...  1.0340263   0.0045

In [67]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Layer
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = tf.shape(q)[-1]
    scaled = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(d_k, tf.float32))
    if mask is not None:
        scaled += mask
    attention = tf.nn.softmax(scaled, axis=-1)
    values = tf.matmul(attention, v)
    return values, attention

class MultiHeadAttention(Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = Dense(3 * d_model)
        self.linear_layer = Dense(d_model)

    def call(self, x, mask=None):
        batch_size = tf.shape(x)[0]
        sequence_length = tf.shape(x)[1]
        qkv = self.qkv_layer(x)
        qkv = tf.reshape(qkv, (batch_size, sequence_length, self.num_heads, 3 * self.head_dim))
        qkv = tf.transpose(qkv, perm=[0, 2, 1, 3])
        q, k, v = tf.split(qkv, num_or_size_splits=3, axis=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = tf.transpose(values, perm=[0, 2, 1, 3])
        values = tf.reshape(values, (batch_size, sequence_length, self.num_heads * self.head_dim))
        out = self.linear_layer(values)
        return out

num_heads= 8
multihead_attention = MultiHeadAttention(d_model= d_model, num_heads= num_heads)

x = multihead_attention(x= output)
x

<tf.Tensor: shape=(10, 250, 512), dtype=float32, numpy=
array([[[ 0.10430601, -0.15512487, -0.26501533, ..., -0.36567742,
          0.34704858,  0.5278903 ],
        [ 0.10323407, -0.15886356, -0.26329628, ..., -0.3705477 ,
          0.34731573,  0.52886975],
        [ 0.09900612, -0.16176273, -0.26456174, ..., -0.372053  ,
          0.3486197 ,  0.5293998 ],
        ...,
        [ 0.10559104, -0.1868048 , -0.28306225, ..., -0.37313545,
          0.37468496,  0.53041506],
        [ 0.10522281, -0.18412626, -0.28762388, ..., -0.3707921 ,
          0.3798082 ,  0.5360537 ],
        [ 0.10601634, -0.18269604, -0.29249632, ..., -0.37099957,
          0.3845331 ,  0.53912956]],

       [[ 0.10401697, -0.15576619, -0.2661923 , ..., -0.36693528,
          0.34442347,  0.5298821 ],
        [ 0.10243075, -0.15941566, -0.26435962, ..., -0.37132376,
          0.3444742 ,  0.53077674],
        [ 0.09924141, -0.16227666, -0.26582012, ..., -0.37360886,
          0.3454441 ,  0.53071606],
        ...

In [68]:
class LayerNormalization(tf.keras.layers.Layer):
    def __init__(self, parameters_shape, eps=1e-5):
        super(LayerNormalization, self).__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = self.add_weight(
            shape=parameters_shape,
            initializer="ones",
            trainable=True,
            name="gamma"
        )
        self.beta = self.add_weight(
            shape=parameters_shape,
            initializer="zeros",
            trainable=True,
            name="beta"
        )

    def call(self, inputs):
        dims = [-i for i in range(1, len(self.parameters_shape) + 1)]
        mean = tf.reduce_mean(inputs, axis=dims, keepdims=True)
        var = tf.reduce_mean(tf.square(inputs - mean), axis=dims, keepdims=True)
        std = tf.sqrt(var + self.eps)
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out


class PositionwiseFeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = tf.keras.layers.Dense(hidden)
        self.linear2 = tf.keras.layers.Dense(d_model)
        self.relu = tf.keras.layers.ReLU()
        self.dropout = tf.keras.layers.Dropout(rate=drop_prob)

    def call(self, x, training=False):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x, training=training)
        x = self.linear2(x)
        return x


class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = tf.keras.layers.Dropout(rate=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = tf.keras.layers.Dropout(rate=drop_prob)

    def call(self, x, self_attention_mask, training=False):
        residual_x = x
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x, training=training)
        x = self.norm1(x + residual_x)
        residual_x = x
        x = self.ffn(x, training=training)
        x = self.dropout2(x, training=training)
        x = self.norm2(x + residual_x)
        return x

num_heads =8
ffn_hidden = 2048
encoder_layer = EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob =0.1)

x = encoder_layer(x,self_attention_mask= None)
x

<tf.Tensor: shape=(10, 250, 512), dtype=float32, numpy=
array([[[ 0.5141616 ,  0.44739604,  0.95789105, ..., -0.38501436,
         -0.29433343,  1.0783138 ],
        [ 0.51609814,  0.44157794,  0.9583447 , ..., -0.39854723,
         -0.28630322,  1.0799603 ],
        [ 0.5111541 ,  0.43939304,  0.9558624 , ..., -0.40295455,
         -0.27511466,  1.0814762 ],
        ...,
        [ 0.5281283 ,  0.39772686,  0.92455465, ..., -0.3841518 ,
         -0.2068237 ,  1.0867946 ],
        [ 0.52891725,  0.40803048,  0.9106763 , ..., -0.38007614,
         -0.19982453,  1.0962558 ],
        [ 0.5351895 ,  0.41350174,  0.8982499 , ..., -0.3815637 ,
         -0.19220492,  1.1017783 ]],

       [[ 0.5161666 ,  0.44461542,  0.95409596, ..., -0.3861025 ,
         -0.30081156,  1.081996  ],
        [ 0.51614887,  0.43954962,  0.9559135 , ..., -0.39906582,
         -0.29413605,  1.0840089 ],
        [ 0.5146126 ,  0.43613815,  0.95089453, ..., -0.40537608,
         -0.2837721 ,  1.0829492 ],
        ...

In [69]:
train_dataset = tf.data.Dataset.from_tensor_slices(valid_english_sentences)
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
iterator = iter(train_dataset)
first_batch = next(iterator)
# first_batch = tf.reshape(first_batch, (1,batch_size))
first_batch = [sentence.numpy().decode('utf-8') for sentence in first_batch]
print(type(first_batch))

<class 'list'>


In [70]:
class SequentialEncoder(tf.keras.Sequential):
    def call(self, inputs, self_attention_mask, training=False):
        x = inputs
        for module in self.layers:
            x = module(x, self_attention_mask, training=training)
        return x


class Encoder(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Encoder, self).__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder([EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def call(self, x, self_attention_mask, start_token, end_token, training=False):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask, training=training)
        return x
drop_prob =0.1
encoder = Encoder(d_model =d_model,ffn_hidden=  ffn_hidden,num_heads= num_heads, drop_prob= drop_prob, num_layers= 1, max_sequence_length= max_sequence_length,
                  language_to_index= english_to_index, START_TOKEN= START_TOKEN, END_TOKEN= END_TOKEN, PADDING_TOKEN= PADDING_TOKEN)
x = encoder(x= first_batch, self_attention_mask= None, start_token= True, end_token= True)
x

<tf.Tensor: shape=(32, 250, 512), dtype=float32, numpy=
array([[[-0.6177005 ,  0.74857634, -0.9057464 , ...,  0.82537603,
         -0.9960935 ,  0.5560274 ],
        [ 0.85070515, -0.13362451,  0.32034773, ...,  0.5841383 ,
         -0.97704524,  0.4606448 ],
        [ 1.1444985 , -1.8249949 ,  0.5708078 , ...,  0.413912  ,
         -0.98397046,  0.5712344 ],
        ...,
        [ 1.6469132 , -0.6939828 , -0.6188047 , ...,  0.5604304 ,
         -0.9067436 ,  0.5719796 ],
        [ 0.8337428 , -1.3771899 ,  0.3314194 , ...,  0.42468733,
         -0.7729402 ,  0.74965775],
        [-0.36150673, -1.0594146 ,  0.9373097 , ...,  0.35746846,
         -0.5790491 ,  0.97050303]],

       [[-0.62151307,  0.74753827, -0.9074398 , ...,  0.82512015,
         -0.9949043 ,  0.55768436],
        [ 0.8235764 , -0.24716254,  0.4228808 , ...,  0.6315357 ,
         -1.0490125 ,  0.5826833 ],
        [ 1.0981297 , -1.7522687 ,  0.4499316 , ...,  0.2865526 ,
         -0.9363413 ,  0.44683152],
        ...

In [99]:
class MultiHeadCrossAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadCrossAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = Dense(2 * d_model)
        self.q_layer = Dense(d_model)
        self.linear_layer = Dense(d_model)

    def call(self, x, y, mask):
        batch_size, sequence_length, d_model = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = tf.reshape(kv, (batch_size, sequence_length, self.num_heads, 2 * self.head_dim))
        q = tf.reshape(q, (batch_size, sequence_length, self.num_heads, self.head_dim))
        kv = tf.transpose(kv, perm=[0, 2, 1, 3])
        q = tf.transpose(q, perm=[0, 2, 1, 3])
        k, v = tf.split(kv, 2, axis=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = tf.transpose(values, perm=[0, 2, 1, 3])
        values = tf.reshape(values, (batch_size, sequence_length, d_model))
        out = self.linear_layer(values)
        return out

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(num_heads=num_heads, d_model=d_model)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = Dropout(drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = Dropout(drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = Dropout(drop_prob)

    def call(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y

    def get_sub_layers(self):
        return [self.self_attention, self.encoder_decoder_attention, self.ffn, self.layer_norm1, self.layer_norm2, self.layer_norm3]



class SequentialDecoder(tf.keras.Sequential):
    def call(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for layer in self.layers:
            y = layer(x, y, self_attention_mask, cross_attention_mask)
        return y


class Decoder(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Decoder, self).__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder([DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def call(self, x, y, self_attention_mask=None, cross_attention_mask=None, start_token=True, end_token=True):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

    def get_layers(self):
        return self.layers


class Transformer(tf.keras.layers.Layer):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 hi_vocab_size,
                 english_to_index,
                 hindi_to_index,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super(Transformer, self).__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, hindi_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = Dense(hi_vocab_size)
        self.device = tf.config.experimental.list_physical_devices('GPU')

    def call(self,
             x,
             y,
             encoder_self_attention_mask=None,
             decoder_self_attention_mask=None,
             decoder_cross_attention_mask=None,
             enc_start_token=False,
             enc_end_token=False,
             dec_start_token=False, # We should make this true
             dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out


In [100]:
print(valid_hindi_sentences[:10])
print(valid_english_sentences[:10])

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।', 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को', '8 सितम्\u200dबर, 2016 को माननीय राष्\u200dट्रपति की स्\u200dवीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्\u200dव में आया', 'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की', 'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.', 'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)', 'सेक्टर 55/56 के एसएचओ अरविंद कुमार ने बताया कि इस मामले में आईपीसी की धारा 376-डी (गैंगरेप) के तहत मामला दर्ज कर लिया गया है।', 'आज नई दिल्ली में मीडिया से बातचीत में पार्टी के राज्य 

In [101]:
# import tensorflow as tf

# class TextDataset(tf.data.Dataset):
#     def _generator(english_sentences, hindi_sentences):
#         for eng, hi in zip(english_sentences, hindi_sentences):
#             yield eng, hi

#     def __new__(cls, english_sentences, hindi_sentences):
#         return tf.data.Dataset.from_generator(
#             cls._generator,
#             output_signature=(
#                 tf.TensorSpec(shape=(), dtype=tf.string),
#                 tf.TensorSpec(shape=(), dtype=tf.string)
#             ),
#             args=(english_sentences, hindi_sentences)
#         )


# dataset = TextDataset(valid_english_sentences, valid_hindi_sentences)
# dataset = dataset.batch(batch_size)


# batches_list = []
# for batch in dataset:
#     batch_list = []
#     batch_list_eng = []
#     batch_list_kan = []
#     for eng, kan in zip(batch[0], batch[1]):
#         batch_list_eng.append(eng.numpy().decode("utf-8"))
#         batch_list_kan.append(kan.numpy().decode("utf-8"))
#     batch_list.append(tuple(batch_list_eng))
#     batch_list.append(tuple(batch_list_kan))
#     batches_list.append(batch_list)

# batches_list[0]

In [102]:
# class IndexedTextDataset:
#     def __init__(self, english_sentences, hindi_sentences):
#         self.english_sentences = english_sentences
#         self.hindi_sentences = hindi_sentences

#     def __len__(self):
#         return len(self.english_sentences)

#     def __getitem__(self, idx):
#         return self.english_sentences[idx], self.hindi_sentences[idx]

# def batch_data(dataset, batch_size):
#     """Yield batches of data from the dataset."""
#     num_samples = len(dataset)
#     for start_idx in range(0, num_samples, batch_size):
#         end_idx = min(start_idx + batch_size, num_samples)
#         batch = [dataset[i] for i in range(start_idx, end_idx)]
#         yield batch

# dataset = IndexedTextDataset(valid_english_sentences, valid_hindi_sentences)
# dataset = list(batch_data(dataset, batch_size))
# # Accessing a specific item
# print(dataset[0])


In [103]:
# import tensorflow as tf

# class TextDataset(tf.data.Dataset):
#     def _generator(english_sentences, kannada_sentences):
#         for eng, kan in zip(english_sentences, kannada_sentences):
#             yield eng, kan

#     def __new__(cls, english_sentences, kannada_sentences):
#         return tf.data.Dataset.from_generator(
#             cls._generator,
#             output_signature=(
#                 tf.TensorSpec(shape=(), dtype=tf.string),
#                 tf.TensorSpec(shape=(), dtype=tf.string)
#             ),
#             args=(english_sentences, kannada_sentences)
#         )

# # Example sentences (you can replace these with your actual sentences)
# english_sentences = ["Hello", "How are you?", "Good morning", "This is a test", "Hope you're well"] * 10  # Repeat to have enough for batching
# kannada_sentences = ["ನಮಸ್ಕಾರ", "ನೀವು ಹೇಗಿದ್ದೀರಾ?", "ಶುಭೋದಯ", "ಇದು ಪರೀಕ್ಷೆ", "ನೀವು ಚೆನ್ನಾಗಿದ್ದೀರಾ ಎಂದು ಭಾವಿಸುತ್ತೇವೆ"] * 10  # Repeat to have enough for batching

# dataset = TextDataset(english_sentences, kannada_sentences)

# # Batch the dataset
# batch_size = 32
# dataset = dataset.batch(batch_size)

# # Example usage: Print the first batch
# for batch in dataset.take(1):
#     for eng, kan in zip(batch[0].numpy(), batch[1].numpy()):
#         print(f'English: {eng.decode("utf-8")}, Kannada: {kan.decode("utf-8")}')


In [104]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
kn_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)


In [117]:
# Define loss function with ignore_index equivalent
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

# Define a custom loss function to ignore padding tokens
def loss_function(real, pred):
    mask = tf.math.not_equal(real, hindi_to_index[PADDING_TOKEN])
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

initializer = tf.keras.initializers.GlorotUniform()

# Iterate over the model's layers and initialize weights with Xavier uniform initializer
# for layer in transformer.encoder.layer.layers:
#     if isinstance(layer, tf.keras.layers.Dense):
#         layer.kernel_initializer = initializer
#         # To reinitialize the kernel with the new initializer
#         layer.build(layer.input_shape)

# for layer in transformer.decoder.layer.layers:
#     if isinstance(layer, tf.keras.layers.Dense):
#         layer.kernel_initializer = initializer
#         # To reinitialize the kernel with the new initializer
#         layer.build(layer.input_shape)


# Initialize layers in the decoder
# for decoder_layer in transformer.decoder.layers:
#     if isinstance(decoder_layer, DecoderLayer):
#         for sub_layer in decoder_layer.get_sub_layers():
#             if isinstance(sub_layer, tf.keras.layers.Dense):
#                 sub_layer.kernel_initializer = initializer
#                 # Rebuild the layer to apply the new initializer
#                 sub_layer.build(sub_layer.input_shape)

# Initialize layers in the encoder (assuming similar structure)
# for layer in transformer.encoder.get_layers():
#     if isinstance(layer, DecoderLayer):
#         for sub_layer in layer.get_sub_layers():
#             if isinstance(sub_layer, tf.keras.layers.Dense):
#                 sub_layer.kernel_initializer = initializer
#                 sub_layer.build(sub_layer.input_shape)  # Rebuild the layer to apply the new initializer

type(transformer.decoder.layers)


__main__.SequentialDecoder